In [7]:
import pandas as pd

# Load the data from the CSV file
data = pd.read_csv("../data/active_construction.csv")

# Grouping by Postcode and summing the Construction Award values
postcode_totals = data.groupby('Postcode')['Construction Award'].sum().sort_values(ascending=False)

# Taking the top 10 postcodes
top_10_postcodes = postcode_totals.head(10)

# Formatting the output
header = "Zip Code             Value of Prime Construction Contract"
print(header)
print("-" * len(header))

for postcode, value in top_10_postcodes.items():
    formatted_value = f"${value:,.2f}"
    print(f"{int(postcode):<20} {formatted_value}")

Zip Code             Value of Prime Construction Contract
---------------------------------------------------------
10314                $5,368,980,925.42
11204                $5,310,136,935.82
11213                $4,814,331,401.32
11225                $4,797,602,607.82
10301                $4,755,078,852.75
10458                $4,083,043,372.05
10467                $3,563,854,007.44
11377                $2,187,021,359.92
11368                $1,929,759,861.36
11355                $1,582,741,953.82


In [9]:
import pandas as pd
import folium

# Load the data
data = pd.read_csv("../data/active_construction.csv")

# Group by Postcode and get total award and average latitude/longitude
grouped = data.groupby('Postcode').agg({
    'Construction Award': 'sum',
    'Latitude': 'mean',
    'Longitude': 'mean'
}).reset_index()

# Create a base map
m = folium.Map(location=[grouped['Latitude'].mean(), grouped['Longitude'].mean()], zoom_start=10)

# Add award values as circle markers to the map
for _, row in grouped.iterrows():
    folium.CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=5,  # or some function of 'Construction Award' if you want variable size
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=f"Zip: {int(row['Postcode'])}<br>Award: ${row['Construction Award']:,}"  # Showing zip and award on click
    ).add_to(m)

m.save('construction_zipcodes.html')  # This saves the map as an interactive HTML file


In [14]:
import folium
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

# Load the construction data
data = pd.read_csv("../data/active_construction.csv")

# Filter for records in postal code 11231
filtered_data = data[data['Postcode'] == 11231].copy()

# Function to snap to grid
GRID_SIZE = 0.001  # e.g., ~100 meters depending on location

def snap_to_grid(coordinate):
    """Snap a latitude or longitude to the nearest grid coordinate"""
    return round(coordinate / GRID_SIZE) * GRID_SIZE

# Snap data to grid
filtered_data['GRID_LAT'] = filtered_data['Latitude'].apply(snap_to_grid)
filtered_data['GRID_LONG'] = filtered_data['Longitude'].apply(snap_to_grid)

# Group by grid location
grouped_data = filtered_data.groupby(['GRID_LAT', 'GRID_LONG'])['Construction Award'].sum().reset_index(name='award_total')

# Filter to keep things visualizable - adjust this threshold as needed
max_award = grouped_data['award_total'].max()
threshold = max_award * 0.1  # For example, removing grid cells with more than 10% of the max award
grouped_data = grouped_data[grouped_data['award_total'] <= threshold]

# Create the map
m = folium.Map(location=[filtered_data['Latitude'].mean(), filtered_data['Longitude'].mean()], zoom_start=15, tiles='CartoDB Positron')

# Calculate max award for scaling purposes
max_award = grouped_data['award_total'].max()

# Adding circles to the map
for idx, row in grouped_data.iterrows():
    # Scale the radius according to award
    radius = (row['award_total'] / max_award) * 20  # This scales the highest award to a radius of 20; adjust as necessary
    
    folium.CircleMarker(
        location=(row['GRID_LAT'], row['GRID_LONG']),
        radius=radius,
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(m)

# Display the map
m

# Save the map to an HTML file if needed
m.save('construction_awards_11231.html')

In [16]:
import folium
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

# Load the construction data
data = pd.read_csv("../data/active_construction.csv")

# Function to snap to grid
GRID_SIZE = 0.001  # e.g., ~100 meters depending on location

def snap_to_grid(coordinate):
    """Snap a latitude or longitude to the nearest grid coordinate"""
    return round(coordinate / GRID_SIZE) * GRID_SIZE

# Snap data to grid
# Drop rows where Latitude or Longitude is NaN
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Now, apply the snap_to_grid function
data['GRID_LAT'] = data['Latitude'].apply(snap_to_grid)
data['GRID_LONG'] = data['Longitude'].apply(snap_to_grid)

# Group by grid location
grouped_data = data.groupby(['GRID_LAT', 'GRID_LONG'])['Construction Award'].sum().reset_index(name='award_total')

# Filter to keep things visualizable - adjust this threshold as needed
max_award = grouped_data['award_total'].max()
threshold = max_award * 0.1  # For example, removing grid cells with more than 10% of the max award
grouped_data = grouped_data[grouped_data['award_total'] <= threshold]

# Create the map centered around the mean latitude and longitude values of the entire dataset
m = folium.Map(location=[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=10, tiles='CartoDB Positron')

# Calculate max award for scaling purposes
max_award = grouped_data['award_total'].max()

# Adding circles to the map
for idx, row in grouped_data.iterrows():
    # Scale the radius according to award
    radius = (row['award_total'] / max_award) * 20  # This scales the highest award to a radius of 20; adjust as necessary
    
    folium.CircleMarker(
        location=(row['GRID_LAT'], row['GRID_LONG']),
        radius=radius,
        fill=True,
        fill_opacity=0.7,
        color='blue',
        fill_color='blue'
    ).add_to(m)

# Display the map
m

# Save the map to an HTML file if needed
m.save('construction_awards_citywide.html')